#### Global Variables

In [1]:
drive_dir = "/content/drive/MyDrive/UNICAP/TCC/research"
work_dir = f"{drive_dir}/results"
data_dir = f"{drive_dir}/datasets"
checkpoints_dir = f"{drive_dir}/checkpoints"
datasets = ["sard"]
checkpoints = []

## Setup Environment

In [2]:
from google.colab import drive
import shutil, errno
import fileinput
import json
import re
import os

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!git clone https://github.com/AFKaro/SARDeep.git

Cloning into 'SARDeep'...
remote: Enumerating objects: 373, done.
remote: Counting objects: 100% (373/373), done.
remote: Compressing objects: 100% (244/244), done.
remote: Total 373 (delta 169), reused 305 (delta 106), pack-reused 0
Receiving objects: 100% (373/373), 76.33 KiB | 12.72 MiB/s, done.
Resolving deltas: 100% (169/169), done.


In [4]:
!pip install optimum[neural-compressor]
!pip install tqdm
!pip install scipy
!pip install terminaltables
!pip install six
!pip install pycocotools
!pip install tensorboard
!pip install opencv-python

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.7/243.7 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 23.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 105.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 40.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 5.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.6/73.6 kB 10.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 62.5 MB/s e

In [5]:
!apt update
!apt install ffmpeg libsm6 libxext6 git ninja-build libglib2.0-0 libsm6 libxrender-dev libxext6
!apt clean
!rm -rf /var/lib/apt/lists/*

Get:1 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease [3,622 B]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Get:3 http://security.ubuntu.com/ubuntu focal-security InRelease [114 kB]
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Get:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease [114 kB]
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Get:9 http://security.ubuntu.com/ubuntu focal-security/main amd64 Packages [2,674 kB]
Get:10 http://archive.ubuntu.com/ubuntu focal-backports InRelease [108 kB]
Hit:11 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Get:12 http://archive.ubuntu.com/ubuntu focal-updates/universe amd64 Packages [1,341 kB]
Hit:13 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Get:14 http://ar

In [6]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117
!pip install openmim
!pip install mmcv-full==1.7.0 -f https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
!cd /content/SARDeep/src && git clone -b 2.x --single-branch https://github.com/open-mmlab/mmdetection.git
%cd /content/SARDeep/src/mmdetection
!pip install -e .

Looking in indexes: https://download.pytorch.org/whl/cu117, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 64.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 92.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 99.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.2/4.2 MB 53.5 MB/s eta 0:00:00
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.0.1+cu118
    Uninstalling torchaudio-2.0.1+cu118:
      Successfully uninstalled torchaudio-2.0.1+cu118
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.3/51.3 kB 5.3 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13/index.html
     ━━━━━━━━━━━━━━━

In [7]:
def copy(src, dst):
    if not os.path.isdir(dst):
      try:
          shutil.copytree(src, dst)
      except OSError as exc: # python >2.5
          if exc.errno in (errno.ENOTDIR, errno.EINVAL):
              shutil.copy(src, dst)
          else: raise

for dataset in datasets:
  copy(f"{data_dir}/{dataset}", f"/content/SARDeep/src/mmdetection/data/{dataset}")

copy(checkpoints_dir, f"/content/SARDeep/src/infra/checkpoints")

In [8]:
!python /content/SARDeep/src/infra/scripts/config_classes.py

/content/drive/MyDrive/UNICAP/TCC/research/datasets sard
Classes: ['person']


In [9]:
for checkpoint in checkpoints:
  !python /content/SARDeep/src/infra/scripts/download_weights.py --weights $checkpoint

## Pipeline

In [10]:
model_name = "faster_rcnn" #@param {type:"string"}
step = "Train" #@param ["All", "Train", "Quantization", "Test"]

with open("/content/SARDeep/resource/pipeline.json", "r") as jsonFile:
    data = json.load(jsonFile)

data["work_dir"] = f"../../{work_dir}"
data["models"] = [model_name]

with open("/content/SARDeep/resource/pipeline.json", "w") as jsonFile:
    json.dump(data, jsonFile)


In [ ]:
!python /content/SARDeep/src/main.py --step $step